In [24]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
import math

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# Visualization tools
import torchvision
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

In [20]:
class TimeSeriesDataset(Dataset):
    def __init__(self, dataframe):
        self.X = torch.tensor(dataframe.iloc[:, :1].values, dtype=torch.float32)  # Input: [Batch, 3]
        self.Y = torch.tensor(dataframe.iloc[:, 1:].values, dtype=torch.float32).unsqueeze(-1) # Output: [Batch, 255, 1]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [21]:
# Positional Encoding for Time Steps
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)  # Add batch dimension

    def forward(self, x):
        return x + self.encoding[:, :x.size(1), :].to(x.device)

# Transformer Model
class TransformerTimeSeriesModel(nn.Module):
    def __init__(self, input_dim, output_dim, seq_length, d_model, nhead, num_layers, dim_feedforward):
        super(TransformerTimeSeriesModel, self).__init__()
        self.input_dim = input_dim
        self.d_model = d_model
        self.seq_length = seq_length
 
        # Input Encoder (maps input to d_model size)
        self.encoder = nn.Linear(input_dim, d_model)  # (Batch, 3) -> (Batch, d_model)
        
        # Project input to match the sequence length
        self.expand_input = nn.Linear(d_model, seq_length * d_model)  # (Batch, d_model) -> (Batch, seq_length * d_model)
        
        # Target embedding for decoder input
        self.target_embedding = nn.Linear(1, d_model)  # New embedding layer for target sequence
  
        # Positional Encoding for Time Steps
        self.pos_encoder = PositionalEncoding(d_model, seq_length)
        
        # Transformer Decoder
        decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        
        # Final Output Layer
        self.output_layer = nn.Linear(d_model, output_dim)  # (Batch, 255, d_model) -> (Batch, 255, 1)

    def forward(self, x, target_seq):
        # x: Input features [Batch, 3]
        # target_seq: Target sequence for teacher forcing [Batch, 255, 1]
        
        # Encode input features
        encoded_input = self.encoder(x)  # [Batch, d_model]
        
        # Expand input to match sequence length
        expanded_input = self.expand_input(encoded_input)  # [Batch, seq_length * d_model]
        expanded_input = expanded_input.view(-1, self.seq_length, self.d_model)  # Reshape to [Batch, 255, d_model]
        
        # Add Positional Encoding
        expanded_input = self.pos_encoder(expanded_input)
        
        # Process the target sequence through the same encoding pipeline
  #      target_embeddings = self.encoder(target_seq)
  #      target_embeddings = nn.Linear(1, d_model)(target_seq)  # [Batch, 255, d_model]
        target_embeddings = self.target_embedding(target_seq)  # [Batch, 255, d_model]
        target_embeddings = self.pos_encoder(target_embeddings)
        
        # Decode sequence
        output = self.transformer_decoder(
            tgt=target_embeddings, memory=expanded_input
        )  # Output shape: [Batch, 255, d_model]
        
        # Map to output dimensions
        predictions = self.output_layer(output)  # [Batch, 255, 1]
        return predictions

In [22]:
def train_model(model, dataloader, optimizer, loss_fn, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        for batch in dataloader:
            x, y = batch  # x: [Batch, N], y: [Batch, T]
            x, y = x.to(device), y.to(device)
            
            # Prepare target for teacher forcing
            target_seq = y 
            #target_seq = y[:, :-1]  # All except last time step
            #actual = y[:, 1:]       # All except first time step
            
            # Forward pass
            output = model(x, target_seq)
            loss = loss_fn(output, y)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

In [34]:
# Load the CSV file
data_input = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_inputs.csv", sep=" ", header=None)
data_output = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_outputs_poor.csv", header=None)
data = pd.concat([data_input, data_output], axis=1)
scaler = MinMaxScaler()
scaler.fit(data)
data = scaler.transform(data)
data = pd.DataFrame(data)
# Split the data into training and validation sets
train_data, valid_data = train_test_split(data, test_size=0.2, random_state=42)

# Save the validation set to a new CSV file
valid_data.to_csv("validation_set.csv", index=False)

In [35]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
75220,0.848036,0.0,0.0,0.0,0.125,0.083333,0.176471,0.318182,0.250000,0.272727,...,0.104046,0.109195,0.108571,0.104046,0.097701,0.091429,0.090909,0.093023,0.092486,0.091954
48955,0.629694,0.0,0.0,0.0,0.000,0.083333,0.117647,0.318182,0.285714,0.333333,...,0.398844,0.385057,0.405714,0.393064,0.419540,0.405714,0.403409,0.406977,0.393064,0.373563
44966,0.574623,0.0,0.0,0.0,0.000,0.083333,0.176471,0.363636,0.357143,0.303030,...,0.450867,0.454023,0.497143,0.485549,0.465517,0.462857,0.448864,0.453488,0.462428,0.459770
13568,0.244526,0.0,0.0,0.0,0.125,0.083333,0.176471,0.363636,0.321429,0.545455,...,0.826590,0.839080,0.851429,0.867052,0.844828,0.828571,0.829545,0.837209,0.826590,0.804598
92727,0.769596,0.0,0.0,0.0,0.250,0.166667,0.294118,0.318182,0.464286,0.363636,...,0.213873,0.212644,0.205714,0.202312,0.195402,0.205714,0.193182,0.186047,0.179191,0.183908


In [36]:
# Load your DataFrame (assuming it's named `df`)
dataset = TimeSeriesDataset(train_data)

In [37]:
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [38]:
# Model parameters
input_dim = 1      # Number of input features
output_dim = 1     # Predicting one value per time step
seq_length = 101   # Number of time steps in output
d_model = 128      # Embedding dimension for Transformer
nhead = 4          # Number of attention heads
num_layers = 2     # Number of Transformer layers
dim_feedforward = 512  # Feedforward network size

# Instantiate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerTimeSeriesModel(
    input_dim, output_dim, seq_length, d_model, nhead, num_layers, dim_feedforward
).to(device)


In [39]:
# Optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()  # Regression loss

# Training loop
num_epochs = 40  # Adjust based on dataset size and performance
train_model(model, dataloader, optimizer, loss_fn, num_epochs, device)

Epoch 1/40, Loss: 0.00033997453283518553
Epoch 2/40, Loss: 0.00011805783287854865
Epoch 3/40, Loss: 0.0003187482652720064
Epoch 4/40, Loss: 1.21088760351995e-05
Epoch 5/40, Loss: 0.0001723524328554049
Epoch 6/40, Loss: 2.203959957114421e-05
Epoch 7/40, Loss: 3.7507875276787672e-06
Epoch 8/40, Loss: 2.5105339318542974e-06
Epoch 9/40, Loss: 0.0001484306703787297
Epoch 10/40, Loss: 2.0865430997218937e-05
Epoch 11/40, Loss: 4.1838011384243146e-06
Epoch 12/40, Loss: 1.7407292034476995e-06
Epoch 13/40, Loss: 1.4382562767423224e-05
Epoch 14/40, Loss: 4.368371264718007e-06
Epoch 15/40, Loss: 1.4602462670154637e-06
Epoch 16/40, Loss: 1.5321724049499608e-06
Epoch 17/40, Loss: 1.483291953263688e-06
Epoch 18/40, Loss: 1.4466116908806725e-06
Epoch 19/40, Loss: 4.437789584699203e-07
Epoch 20/40, Loss: 1.9874409190379083e-05
Epoch 21/40, Loss: 2.8447507247619797e-06
Epoch 22/40, Loss: 1.6456745015602792e-06
Epoch 23/40, Loss: 6.857779339952685e-07
Epoch 24/40, Loss: 5.731689043386723e-07
Epoch 25/40,

In [40]:
with open('bank_reserves_inputs_surr.csv', 'w', newline='') as bank_reserves_inputs, \
    open('bank_reserves_outputs_poor_surr.csv', 'w', newline='') as bank_reserves_outputs_poor:
    for i in np.arange(100000):
        reserve_perc = random.uniform(0,100)
        bankRes = bankReservesModel.BankReserves(init_people=500, rich_threshold=10, reserve_percent=reserve_perc)
        bankRes.run_model()
        results = bankRes.datacollector.get_model_vars_dataframe()
        print(reserve_perc, file=bank_reserves_inputs)
        print(*results['Rich'].to_list(), file=bank_reserves_outputs_rich, sep=",")
        print(*results['Middle Class'].to_list(), file=bank_reserves_outputs_middle, sep=",")
        print(*results['Poor'].to_list(), file=bank_reserves_outputs_poor, sep=",")
bank_reserves_inputs.close()
bank_reserves_outputs_poor.close()

TransformerTimeSeriesModel(
  (encoder): Linear(in_features=1, out_features=128, bias=True)
  (expand_input): Linear(in_features=128, out_features=12928, bias=True)
  (target_embedding): Linear(in_features=1, out_features=128, bias=True)
  (pos_encoder): PositionalEncoding()
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1

In [48]:
model.forward(0.10, 101)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not float